In [1]:
import tensorflow as tf
import pathlib
import numpy as np
import os

# Constants
CHARSET = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
NUM_CLASSES = len(CHARSET) + 1  # 36 chars + blank=0
IMG_WIDTH = 94
IMG_HEIGHT = 24
BATCH_SIZE = 32
MAX_LABEL_LEN = 10

# Global lookup table (shift by +1, so blank=0)
keys = tf.constant(list(CHARSET))
values = tf.constant(list(range(1, len(CHARSET)+1)), dtype=tf.int32)  # 1..36
char_to_num_table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(keys, values),
    default_value=0
)

# Reverse table for decoding
num_to_char_table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(values, keys),
    default_value=''
)

# Extract label from filename
def process_filename(file_path):
    filename = tf.strings.split(file_path, os.sep)[-1]
    filename = tf.strings.regex_replace(filename, r'\.[^.]+$', '')
    filename = tf.strings.regex_replace(filename, r'_.*$', '')
    return filename

# Encode label with padding using 0 as blank
def encode_label(label):
    chars = tf.strings.unicode_split(label, 'UTF-8')
    indices = tf.cast(char_to_num_table.lookup(chars), tf.int32)
    pad_len = tf.maximum(0, MAX_LABEL_LEN - tf.shape(indices)[0])
    indices = tf.pad(indices, [[0, pad_len]], constant_values=0)[:MAX_LABEL_LEN]
    return indices

# Load and preprocess image
def load_and_preprocess_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    
    label = process_filename(file_path)
    encoded_label = encode_label(label)
    return image, encoded_label

# Create dataset
def create_dataset(folder_path, shuffle=True):
    files = tf.data.Dataset.list_files(str(folder_path / '*.*'), shuffle=shuffle)
    dataset = files.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

# Load datasets
train_dir = pathlib.Path("lprds/train")
val_dir = pathlib.Path("lprds/val")
test_dir = pathlib.Path("lprds/test")

train_ds = create_dataset(train_dir, shuffle=True)
val_ds = create_dataset(val_dir, shuffle=False)
test_ds = create_dataset(test_dir, shuffle=False)

# Inspect dataset
def inspect_dataset(dataset, name, num_samples=5):
    print(f"\n{name} dataset:")
    for images, labels in dataset.take(1):
        print(f"Images: {images.shape}, Labels: {labels.shape}")
        for i in range(min(num_samples, BATCH_SIZE)):
            print(f"Sample {i+1}: {labels[i].numpy()}, length: {np.sum(labels[i].numpy() != 0)}")

inspect_dataset(train_ds, "Train")
inspect_dataset(val_ds, "Validation")
inspect_dataset(test_ds, "Test")



Train dataset:
Images: (32, 24, 94, 3), Labels: (32, 10)
Sample 1: [17  2  2  8  7  0  0  0  0  0], length: 5
Sample 2: [14 27  1  5  1 32 26  0  0  0], length: 7
Sample 3: [23 17  1  7  5  4  9  0  0  0], length: 7
Sample 4: [23 32 22  5  3  3  0  0  0  0], length: 6
Sample 5: [14 29  1  6  2 13 14  0  0  0], length: 7

Validation dataset:
Images: (32, 24, 94, 3), Labels: (32, 10)
Sample 1: [ 2 27 11 16  1  6  1  0  0  0], length: 7
Sample 2: [ 4  6  4 11 29 21  4  6  0  0], length: 8
Sample 3: [ 5  2 20 11  4  5  0  0  0  0], length: 6
Sample 4: [ 6  2  3  7 18 32 22  0  0  0], length: 7
Sample 5: [ 7  5  1  3 32 18  3  6  0  0], length: 8

Test dataset:
Images: (32, 24, 94, 3), Labels: (32, 10)
Sample 1: [ 2  1  4  4 19 28  0  0  0  0], length: 6
Sample 2: [ 3  4  5  9 34 28  3  6  0  0], length: 8
Sample 3: [ 4  5 15 24 10  2 10 10  0  0], length: 8
Sample 4: [ 4  9  2 11 30 21  9  4  0  0], length: 8
Sample 5: [ 6  6 29 17  6  4  0  0  0  0], length: 6


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

def small_basic_block(Cout, name=None):
    mid = Cout // 4
    seq = models.Sequential(name=name)
    seq.add(layers.Conv2D(mid, kernel_size=1, strides=1, padding="same"))
    seq.add(layers.BatchNormalization())
    seq.add(layers.ReLU())
    
    seq.add(layers.Conv2D(mid, kernel_size=(3,1), strides=1, padding="same")) # height=3
    seq.add(layers.BatchNormalization())
    seq.add(layers.ReLU())

    seq.add(layers.Conv2D(mid, kernel_size=(1,3), strides=1, padding="same")) # width=3
    seq.add(layers.BatchNormalization())
    seq.add(layers.ReLU())

    seq.add(layers.Conv2D(Cout, kernel_size=1, strides=1, padding="same"))
    seq.add(layers.BatchNormalization())
    seq.add(layers.ReLU())

    return seq

def global_context_block(x, num_classes, gc_dim=128, name=None):
    """Global context embedding as in LPRNet paper."""
    B, H, W, C = x.shape

    # Step 1: Global context vector from backbone
    context = layers.Flatten(name=f"{name}_flatten")(x)             # (B, H*W*C)
    context = layers.Dense(gc_dim, activation='relu', name=f"{name}_fc")(context)  # (B, gc_dim)

    # Step 2: Tile back to spatial map
    context = layers.RepeatVector(H * W, name=f"{name}_repeat")(context)  # (B, H*W, gc_dim)
    context = layers.Reshape((H, W, gc_dim), name=f"{name}_reshape")(context)  # (B, H, W, gc_dim)

    # Step 3: Concatenate with backbone features
    x = layers.Concatenate(axis=-1, name=f"{name}_concat")([x, context])  # (B, H, W, C+gc_dim)

    # Step 4: 1×1 Conv to adjust channels → num_classes
    x = layers.Conv2D(num_classes, (1,1), strides=1, padding="same", name=f"{name}_conv1x1")(x)

    return x


def LPRNet(num_classes=37, dropout_rate=0.5):
    inputs = layers.Input(shape=(24,94,3), name="input") # HxWxD

    # Backbone
    x = layers.Conv2D(64, (3,3), strides=1, padding="valid")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.MaxPooling2D((3,3), strides=(1,1), padding="valid")(x)
    x = small_basic_block(128,"sbb1")(x)
    
    x = layers.MaxPooling2D((3,3), strides=(2,1), padding="valid")(x) # 64? it should 128 instead?

    x = small_basic_block(256,"sbb2")(x)
    x = small_basic_block(256,"sbb3")(x)

    x = layers.MaxPooling2D((3,3), strides=(2,1), padding="valid")(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Conv2D(256,(1,4), strides=1, padding="valid")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Conv2D(num_classes,(1, 13), strides=1, padding="valid")(x)
    # Add global context
    x = global_context_block(x, num_classes=num_classes, gc_dim=128, name="gc")

    return models.Model(inputs=inputs, outputs=x, name="LPRNet")

# ---- Test ----
model = LPRNet(num_classes=37)
model.summary()

dummy = tf.random.normal((1,24,94,3))
out = model(dummy)
print("Output shape:", out.shape)


Model: "LPRNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 24, 94, 3) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 22, 92,    │      1,792 │ input[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 22, 92,    │        256 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_5 (ReLU)      │ (None, 22, 92,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 20, 90,    │          0 │ re_lu_5[0][0]     │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sbb1 (Sequential)   │ (None, 20, 90,    │     13,408 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 9, 88,     │          0 │ sbb1[0][0]        │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sbb2 (Sequential)   │ (None, 9, 88,     │     51,392 │ max_pooling2d_3[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sbb3 (Sequential)   │ (None, 9, 88,     │     59,584 │ sbb2[0][0]        │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 4, 86,     │          0 │ sbb3[0][0]        │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 4, 86,     │          0 │ max_pooling2d_4[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 4, 83,     │    262,400 │ dropout[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 83,     │      1,024 │ conv2d_18[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_18 (ReLU)     │ (None, 4, 83,     │          0 │ batch_normalizat… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 4, 83,     │          0 │ re_lu_18[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 4, 71, 37) │    123,173 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gc_flatten          │ (None, 10508)     │          0 │ conv2d_19[0][0]   │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,864,323 (7.11 MB)

 Trainable params: 1,861,443 (7.10 MB)

 Non-trainable params: 2,880 (11.25 KB)

Output shape: (1, 4, 71, 37)


In [ ]:
import tensorflow as tf

# Assuming the LPRNet model is already defined as in your code.
# Note: There's a typo in your model code - "SmallBasicBlock" should be "small_basic_block" (case-sensitive).
# Correct it to small_basic_block(256, "sbb2")(x) and small_basic_block(256, "sbb3")(x)

# Custom CTC loss function to handle variable label lengths
def ctc_loss(y_true, y_pred):
    # y_pred shape: (B, 4, 71, 37)
    # Reduce mean over height to get (B, 71, 37) for CTC (time steps along width)
    y_pred = tf.reduce_mean(y_pred, axis=1)
    
    # Cast y_true to int32 if not already
    y_true = tf.cast(y_true, tf.int32)
    
    # Compute label lengths: number of non-zero elements (since 0 is pad)
    label_length = tf.math.count_nonzero(y_true, axis=1, dtype=tf.int32)
    
    # Logit lengths: full time steps for each sample
    batch_size = tf.shape(y_pred)[0]
    logit_length = tf.fill([batch_size], tf.shape(y_pred)[1])
    
    # Compute CTC loss using tf.nn.ctc_loss
    loss = tf.nn.ctc_loss(
        labels=y_true,
        logits=y_pred,
        label_length=label_length,
        logit_length=logit_length,
        logits_time_major=False,
        blank_index=0  # Assuming blank is class 0, characters are 1-36
    )
    
    # Return mean loss
    return tf.reduce_mean(loss)


In [7]:
from tensorflow.keras import optimizers, callbacks

# Load the saved model if resuming, otherwise instantiate a new one
try:
    model = tf.keras.models.load_model(
        "lprnet_checkpoint.keras",
        custom_objects={"ctc_loss": ctc_loss},
        compile=False
    )
    print("Loaded model from checkpoint.")
except:
    print("No checkpoint found, creating new model.")
    model = LPRNet(num_classes=37, dropout_rate=0.5)  # Assuming LPRNet is defined

# Compile the model with Adam optimizer
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=ctc_loss
)

# Define callbacks
checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="lprnet_checkpoint.keras",
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    verbose=1
)


reduce_lr_callback = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,  # Reduce learning rate by half
    patience=5,  # Wait 5 epochs before reducing
    min_lr=1e-6,  # Minimum learning rate
    verbose=1
)

# Train the model with callbacks
history = model.fit(
    train_ds,  # Assuming train_ds is defined
    validation_data=val_ds,  # Assuming val_ds is defined
    epochs=100,
    initial_epoch=0,  # Set to the epoch you want to start from if resuming
    callbacks=[checkpoint_callback, reduce_lr_callback],
    verbose=1
)

# Evaluate on test dataset
test_loss = model.evaluate(test_ds)  # Assuming test_ds is defined
print(f"Test Loss: {test_loss}")

# Save the final model
model.save("lprnet_model_final.keras")

Loaded model from checkpoint.
Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 774ms/step - loss: 1.1617
Epoch 1: val_loss improved from None to 21.04774, saving model to lprnet_checkpoint.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 25s 867ms/step - loss: 1.3247 - val_loss: 21.0477 - learning_rate: 0.0010
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 729ms/step - loss: 1.2151
Epoch 2: val_loss improved from 21.04774 to 17.16462, saving model to lprnet_checkpoint.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 15s 769ms/step - loss: 1.2470 - val_loss: 17.1646 - learning_rate: 0.0010
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 721ms/step - loss: 1.0474
Epoch 3: val_loss improved from 17.16462 to 14.24525, saving model to lprnet_checkpoint.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 15s 760ms/step - loss: 1.0714 - val_loss: 14.2452 - learning_rate: 0.0010
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 730ms/step - loss: 0.8902
Epoch 4: val_loss improved from 14.24525 to 14.04349, saving model to lprnet_checkpoint.keras
19/19 ━━━━━━━━━━━━━━━━━

In [10]:
# Character mapping (adjust based on your dataset)
# Example: 0=blank, 1-10=digits 0-9, 11-36=A-Z
char_map = {0: '', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9',
            11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E', 16: 'F', 17: 'G', 18: 'H', 19: 'I', 20: 'J',
            21: 'K', 22: 'L', 23: 'M', 24: 'N', 25: 'O', 26: 'P', 27: 'Q', 28: 'R', 29: 'S', 30: 'T',
            31: 'U', 32: 'V', 33: 'W', 34: 'X', 35: 'Y', 36: 'Z'}

# Function to preprocess a single image
def preprocess_image(image_path):
    # Load image (assuming it's a file path; adjust if image is a numpy array)
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)  # Normalize to [0, 1]
    img = tf.image.resize(img, [24, 94])  # Resize to match model input
    img = tf.expand_dims(img, axis=0)  # Add batch dimension: (1, 24, 94, 3)
    return img

# Function to decode CTC output
def decode_ctc_output(logits, char_map):
    # Reduce mean over height to get (B, 71, 37)
    logits = tf.reduce_mean(logits, axis=1)  # Shape: (1, 71, 37)
    
    # CTC greedy decoder
    decoded, _ = tf.nn.ctc_greedy_decoder(
        inputs=tf.transpose(logits, perm=[1, 0, 2]),  # Time-major: (71, 1, 37)
        sequence_length=tf.ones([1], dtype=tf.int32) * logits.shape[1],
        blank_index=0
    )
    
    # Convert sparse tensor to dense and extract the sequence
    decoded_sequence = tf.sparse.to_dense(decoded[0]).numpy()[0]
    
    # Map indices to characters
    prediction = ''.join([char_map.get(int(idx), '') for idx in decoded_sequence])
    return prediction

# Function to predict license plate from an image
def predict_license_plate(image_path, model_path="lprnet_checkpoint.keras"):
    # Load the model
    model = tf.keras.models.load_model(
        model_path,
        custom_objects={"ctc_loss": ctc_loss},
        compile=False
    )
    
    # Preprocess the image
    img = preprocess_image(image_path)
    
    # Get model prediction
    logits = model(img)  # Shape: (1, 4, 71, 37)
    
    # Decode the prediction
    prediction = decode_ctc_output(logits, char_map)
    
    return prediction

# Example usage
if __name__ == "__main__":
    # Example image path
    image_path = r"lprds\test\8427XX29.jpg"  # Replace with actual image path
    try:
        predicted_plate = predict_license_plate(image_path, model_path="lprnet_checkpoint.keras")
        print(f"Predicted License Plate: {predicted_plate}")
    except Exception as e:
        print(f"Error predicting license plate: {e}")

Predicted License Plate: 8427XX29
